# LLM generation은 별도의 환경을 사용하였습니다. 자세한 내용은 아래 PIP list를 참조해주세요


In [ ]:
# Python version: 3.10.18

In [ ]:
"""
absl-py==2.3.1
accelerate==1.8.1
aiohappyeyeballs==2.6.1
aiohttp==3.12.14
aiosignal==1.4.0
airportsdata==20250706
annotated-types==0.7.0
anyio==4.9.0
astor==0.8.1
async-timeout==5.0.1
attrs==25.3.0
blake3==1.0.5
cachetools==6.1.0
certifi==2025.6.15
charset-normalizer==3.4.2
click==8.2.1
cloudpickle==3.1.1
compressed-tensors==0.10.2
contourpy==1.3.2
cupy-cuda12x==13.5.1
cycler==0.12.1
deepspeed==0.17.1
depyf==0.18.0
dill==0.4.0
diskcache==5.6.3
distro==1.9.0
dnspython==2.7.0
einops==0.8.1
email_validator==2.2.0
exceptiongroup==1.3.0
fastapi==0.116.1
fastapi-cli==0.0.8
fastapi-cloud-cli==0.1.4
fastrlock==0.8.3
filelock==3.18.0
fonttools==4.58.4
frozenlist==1.7.0
fsspec==2025.5.1
gguf==0.17.1
gitdb==4.0.12
GitPython==3.1.44
h11==0.16.0
h5py==3.14.0
hf-xet==1.1.5
hjson==3.1.0
httpcore==1.0.9
httptools==0.6.4
httpx==0.28.1
huggingface-hub==0.33.1
idna==3.10
immutabledict==4.2.1
interegular==0.3.3
Jinja2==3.1.6
jiter==0.10.0
joblib==1.5.1
jsonschema==4.24.0
jsonschema-specifications==2025.4.1
kiwisolver==1.4.8
lark==1.2.2
llguidance==0.7.30
llvmlite==0.44.0
lm-format-enforcer==0.10.11
markdown-it-py==3.0.0
MarkupSafe==3.0.2
matplotlib==3.10.3
mdurl==0.1.2
mistral_common==1.7.0
mpmath==1.3.0
msgpack==1.1.1
msgspec==0.19.0
multidict==6.6.3
nest-asyncio==1.6.0
networkx==3.4.2
ninja==1.11.1.4
nltk==3.9.1
numba==0.61.2
numpy==2.2.6
nvidia-cublas-cu12==12.6.4.1
nvidia-cuda-cupti-cu12==12.6.80
nvidia-cuda-nvrtc-cu12==12.6.77
nvidia-cuda-runtime-cu12==12.6.77
nvidia-cudnn-cu12==9.5.1.17
nvidia-cufft-cu12==11.3.0.4
nvidia-cufile-cu12==1.11.1.6
nvidia-curand-cu12==10.3.7.77
nvidia-cusolver-cu12==11.7.1.2
nvidia-cusparse-cu12==12.5.4.2
nvidia-cusparselt-cu12==0.6.3
nvidia-nccl-cu12==2.26.2
nvidia-nvjitlink-cu12==12.6.85
nvidia-nvtx-cu12==12.6.77
openai==1.90.0
opencv-python-headless==4.12.0.88
ortools==9.14.6206
outlines==0.1.11
outlines_core==0.1.26
packaging==25.0
pandas==2.3.0
partial-json-parser==0.2.1.1.post6
peft==0.16.0
pillow==11.0.0
platformdirs==4.3.8
prometheus-fastapi-instrumentator==7.1.0
prometheus_client==0.22.1
propcache==0.3.2
protobuf==6.31.1
psutil==7.0.0
py-cpuinfo==9.0.0
pybase64==1.4.1
pycountry==24.6.1
pydantic==2.11.7
pydantic_core==2.33.2
Pygments==2.19.2
pyparsing==3.2.3
python-dateutil==2.9.0.post0
python-dotenv==1.1.1
python-json-logger==3.3.0
python-multipart==0.0.20
pytz==2025.2
PyYAML==6.0.2
pyzmq==27.0.0
ray==2.47.1
referencing==0.36.2
regex==2024.11.6
requests==2.32.4
rich==14.0.0
rich-toolkit==0.14.8
rignore==0.6.2
rouge_score==0.1.2
rpds-py==0.26.0
safetensors==0.5.3
scikit-learn==1.7.0
scipy==1.15.3
sentencepiece==0.2.0
sentry-sdk==2.32.0
setproctitle==1.3.6
shellingham==1.5.4
six==1.17.0
smmap==5.0.2
sniffio==1.3.1
starlette==0.47.1
sympy==1.14.0
threadpoolctl==3.6.0
tiktoken==0.9.0
tokenizers==0.21.2
torch==2.7.0
torchaudio==2.7.0
torchvision==0.22.0
tqdm==4.67.1
transformers==4.53.0
triton==3.3.0
typer==0.16.0
typing-inspection==0.4.1
typing_extensions==4.14.0
tzdata==2025.2
urllib3==2.5.0
uvicorn==0.35.0
uvloop==0.21.0
vllm==0.9.2
wandb==0.20.1
watchfiles==1.1.0
websockets==15.0.1
xformers==0.0.30
xgrammar==0.1.19
yarl==1.20.1
"""

In [ ]:
import pandas as pd
from vllm import LLM, SamplingParams
from tqdm import tqdm

llm = LLM(model="meta-llama/Llama-3.1-8B-Instruct",
          tensor_parallel_size=4)

# 샘플링 설정
sampling_params = SamplingParams(
    temperature=0.8,
    top_p=0.9,
    max_tokens=500,  # 다음 문단 길이
    stop=["\n\n"]  # 문단 단위 종료
)

# CSV 불러오기
df = pd.read_csv("../data_oof/train.csv")

print(df.head())

# 다음 문단 예측이 필요한 행만 필터링
df_to_generate = df[df["generated"] == 0].copy()

print(f"총 {len(df_to_generate)}개의 문단 생성 필요")

# 배치 설정
batch_size = 128
generated_paragraphs = [""] * len(df_to_generate)
prompts = []

# 프롬프트 미리 생성
for idx, row in df_to_generate.iterrows():
    input_paragraph = row["full_text"]
    prompt = f"""아래는 하나의 주제에 대해 연속적으로 쓰인 글입니다. 첫 번째 문단을 바탕으로 두 번째 문단을 자연스럽게 이어서 작성하세요. 주제의 흐름을 유지하며 논리적으로 전개해보세요. 문체와 어조는 첫 문단과 일치시키세요.

[첫 번째 문단]
{input_paragraph}

[두 번째 문단]
"""
    prompts.append(prompt)

# 배치 단위 추론 및 저장
for i in tqdm(range(0, len(prompts), batch_size)):
    batch_prompts = prompts[i:i+batch_size]
    outputs = llm.generate(batch_prompts, sampling_params)
    for j, output in enumerate(outputs):
        generated_text = output.outputs[0].text.strip()
        generated_paragraphs[i + j] = generated_text

    # 10,000개마다 중간 저장
    if (i + batch_size) % 10000 < batch_size:
        df_to_generate["generated_paragraph"] = generated_paragraphs
        df_to_generate.iloc[:i + batch_size].to_csv(f"../data_oof_phase3/generated_output_up_to_{i+batch_size}.csv", index=False)

# 최종 저장
# df_to_generate["generated_paragraph"] = generated_paragraphs
# df_to_generate.to_csv("generated_output.csv", index=False)